In [ ]:
# dependencies
import ee
import geemap

ee.Initialize()
Map = geemap.Map()

In [ ]:
# import image and FeatureCollection
raw_countries = ee.FeatureCollection('users/ccbitt23/testDHS')
countries = raw_countries.map(lambda point: point.buffer(10_000))   # expand the points
gfc2019 = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

#select bands
forest_loss = gfc2019.select('loss').multiply(ee.Image.pixelArea()) # in square meters
loss_year = gfc2019.select('lossyear')
big_band = forest_loss.addBands(loss_year)

# calculate numbers
forest_stats = big_band.reduceRegion(
  reducer=ee.Reducer.sum().group(groupField=1, groupName='Year'),   # sum the loss band and group by lossyear 
  geometry=countries,
  scale=30,
  maxPixels=1e9)

client_forest_stats = forest_stats.getInfo()                        # create client object 

In [ ]:
# process the data from GEE into an end product
unpacked = client_forest_stats.get('groups')
desired_year = 2006
desired_index = desired_year - 2001
area_loss = unpacked[desired_index].get('sum')

# print out the output
print(f'The rounded area of forest loss in {desired_year} was {round(area_loss):,} square meters.')
print(f'That\'s {round(area_loss/1e6):,} square kilometers.')